In [1]:
import pandas as pd
#load merged.csv
merged = pd.read_csv(r'C:\Users\zuzan\OneDrive\Desktop\Python Load\Buurt_.csv', sep = ',')
merged.buurtcode = merged.buurtcode.astype(str)
merged = merged[['buurtcode', 'buurtname']]
merged.drop_duplicates(subset ='buurtcode', keep = 'first', inplace = True)
print(merged.shape)
print(merged.head())

(13305, 2)
  buurtcode                                    buurtname
0     30000                           Appingedam-Centrum
1     30001                              Appingedam-West
2     30002                              Appingedam-Oost
3     30007  Verspreide huizen Damsterdiep en Eemskanaal
4     30008  Verspreide huizen ten zuiden van Eemskanaal


In [8]:
import pandas as pd
density = pd.read_csv(r'C:\Users\zuzan\OneDrive\Desktop\repository\product3team9\data\raw\cbs_datasets\cbs_sex_age_populationdensity.csv', 
                     error_bad_lines = False, sep = ';')
density = density[['NBH_code', 'No_residents','Density']]
density.head()

,NBH_code,No_residents,Density
0,NL00,17181084,510
1,GM1680,25390,92
2,WK168000,3560,243
3,BU16800000,3415,1354
4,BU16800009,145,12


In [9]:
density = density[density['NBH_code'].str.contains('BU')]
density['NBH_code'] = density['NBH_code'].astype(str)
for a in range(len(density)):
    if density['NBH_code'].iloc[a][2:3] == '0' and density['NBH_code'].iloc[a][3:4] != '0':
        density['NBH_code'].iloc[a] = ''.join(density['NBH_code'].iloc[a].rsplit('BU0'))
    elif density['NBH_code'].iloc[a][2:4] == '00' and density['NBH_code'].iloc[a][4:5] != '0':
        density['NBH_code'].iloc[a] = ''.join(density['NBH_code'].iloc[a].rsplit('BU00'))
    elif density['NBH_code'].iloc[a][2:5] == '000':
        density['NBH_code'].iloc[a] = ''.join(density['NBH_code'].iloc[a].rsplit('BU000'))
    else:
        density['NBH_code'].iloc[a] = density['NBH_code'].iloc[a][2:]
   
density.head(100)

C:\Users\zuzan\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,NBH_code,No_residents,Density
3,16800000,3415,1354
4,16800009,145,12
6,16800100,1260,738
7,16800109,170,10
9,16800200,325,562
...,...,...,...
134,1970302,2410,1492
135,1970303,1550,2396
136,1970304,2655,2987
137,1970305,825,1971


In [10]:
cleaned_density = pd.merge(density, merged, left_on='NBH_code', right_on='buurtcode', how = 'left')
cleaned_density = cleaned_density.loc[cleaned_density['buurtname'].isna() == False]
cleaned_density = cleaned_density[['buurtname','No_residents','Density']]
cleaned_density.shape

(13305, 3)

In [17]:
#cleaned_density['No_residents']=cleaned_density['No_residents'].str.extract(r'(\d)')
cleaned_density['Density']=cleaned_density['Density'].str.extract(r'(\d)')

In [21]:
cleaned_density = cleaned_density.dropna(subset = ['No_residents', 'Density'])

In [23]:
cleaned_density.isna().sum()

buurtname       0
No_residents    0
Density         0
dtype: int64

In [22]:
cleaned_density['No_residents']=cleaned_density['No_residents'].astype(int)
cleaned_density['Density']=cleaned_density['Density'].astype(int)
print(cleaned_density.dtypes)

buurtname       object
No_residents     int32
Density          int32
dtype: object


<ipython-input-22-db30724e87d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_density['No_residents']=cleaned_density['No_residents'].astype(int)
<ipython-input-22-db30724e87d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_density['Density']=cleaned_density['Density'].astype(int)


In [26]:
cleaned_density=cleaned_density.drop_duplicates(subset='buurtname', keep='first', inplace=False, ignore_index=False)

In [29]:
cleaned_density.head(100)

,buurtname,No_residents,Density
0,Annen,3415,1
1,Verspreide huizen Annen,145,1
2,Eext,1260,7
3,Verspreide huizen Eext,170,1
4,Anloo,325,5
...,...,...,...
101,Dinxperlo Centrum/Oost,2795,3
102,Dinxperlo Noord,1810,2
103,Dinxperlo West,2425,1
104,De Heurne-kern,330,1


In [28]:
cleaned_density.to_csv('cleaned_density.csv', index=False)